In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

/home/villacorta/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/villacorta/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


['CONTENT_CATEGORY.csv',
 '.ipynb_checkpoints',
 'conversiones.csv',
 'sampleSubmission.csv',
 'pageviews.csv',
 'CONTENT_CATEGORY_TOP.csv',
 'device_data.csv',
 'PAGE.csv',
 'SITE_ID.csv',
 'CONTENT_CATEGORY_BOTTOM.csv']

In [2]:
def get_schedule(val):
    if val < 6:
        return 'madrugada'
    elif val < 9:
        return 'antes_del_trabajo'
    elif val < 13:
        return 'trabajo_manana'
    elif val < 16:
        return 'almuerzo'
    elif val < 19:
        return 'trabajo_tarde'
    else:
        return 'luego_del_trabajo'

In [3]:
col_user = 'USER_ID'
data = pd.read_csv(
    "{}/pageviews.csv".format(subfolder), parse_dates=["FEC_EVENT"]
).sort_values(
    [col_user, "FEC_EVENT"]
)
data.shape

(17936934, 8)

In [4]:
del data['CONTENT_CATEGORY_BOTTOM']
data['weekday_view'] = data['FEC_EVENT'].dt.weekday
data['hour'] = data['FEC_EVENT'].dt.hour
data['schedule_view'] = data['hour'].apply(get_schedule)
del data['hour']

In [5]:
cols = list(data.columns)[1:-3] + ['weekday_view', 'schedule_view']
cols

['PAGE',
 'CONTENT_CATEGORY',
 'CONTENT_CATEGORY_TOP',
 'SITE_ID',
 'ON_SITE_SEARCH_TERM',
 'weekday_view',
 'schedule_view']

In [6]:
trimestre_data = data[data.FEC_EVENT.dt.month > 0]

X_test = []

for c in cols:
    print("-->", c)
    temp = pd.crosstab(trimestre_data.USER_ID, trimestre_data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
    del temp
    
X_test = pd.concat(X_test, axis=1)
del trimestre_data

--> PAGE
--> CONTENT_CATEGORY
--> CONTENT_CATEGORY_TOP
--> SITE_ID
--> ON_SITE_SEARCH_TERM
--> weekday_view
--> schedule_view


In [7]:
data = data[data.FEC_EVENT.dt.month < 10]

X_train = []
for c in cols:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
    
X_train = pd.concat(X_train, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM
haciendo weekday_view
haciendo schedule_view


In [8]:
X_train.shape, X_test.shape

((11387, 1945), (11676, 2114))

In [9]:
list(X_test.columns[-10:])

['weekday_view_3',
 'weekday_view_4',
 'weekday_view_5',
 'weekday_view_6',
 'schedule_view_almuerzo',
 'schedule_view_antes_del_trabajo',
 'schedule_view_luego_del_trabajo',
 'schedule_view_madrugada',
 'schedule_view_trabajo_manana',
 'schedule_view_trabajo_tarde']

In [10]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [11]:
y_prev = pd.read_csv("data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(
        set(X_train.index))
y_train.loc[list(idx)] = 1

In [12]:
X_train.shape, X_test.shape, y_train.shape

((11387, 1945), (11676, 1945), (11387,))

In [13]:
convertion = pd.read_csv("{}/conversiones.csv".format(subfolder)).drop_duplicates()[['USER_ID','mes']].sort_values(['USER_ID','mes']).reset_index(drop=True)
for col in convertion.columns:
    convertion[col] = convertion[col].astype(int)

convertion.dtypes
def get_trimestre(val):
    if val <= 3:
        return 1
    elif val <= 6:
        return 2
    elif val <= 9:
        return 3
    elif val <= 12:
        return 4
    return 0

convertion['trimestre'] = convertion['mes'].apply(get_trimestre)
for col in convertion.columns:
    print(col, convertion[col].unique().shape)

del convertion['mes']
convertion.head()


USER_ID (1086,)
mes (12,)
trimestre (4,)


,USER_ID,trimestre
0,13,3
1,22,4
2,40,1
3,57,2
4,92,1


In [14]:

col_user = 'USER_ID'
pageview = pd.read_csv("{}/pageviews.csv".format(subfolder), parse_dates=["FEC_EVENT"]).sort_values([col_user, "FEC_EVENT"])
all_user = set(list(pageview[col_user].unique()))
print(len(all_user))
del pageview

data_conv = []

for val in sorted(list(convertion['trimestre'].unique())):
    col_target = 'TARGET_{}'.format(val)
    conv_trim = convertion[convertion['trimestre'] == val].drop_duplicates().reset_index(drop=True)
    unique_users_conv = set(list(conv_trim[col_user].unique()))
    conv_trim.index = conv_trim[col_user]
    
    conv_trim[col_target] = 1
    
    del conv_trim.index.name
    del conv_trim[col_user]
    del conv_trim['trimestre']

    temp0 = pd.DataFrame(0, index=list(all_user - unique_users_conv), columns=[col_target])
    X_test_intern = pd.concat([conv_trim, temp0]).sort_index()

    print("X_test_intern post:: ", X_test_intern.shape)
    
    data_conv.append(X_test_intern.copy())
    del X_test_intern
    del conv_trim


data_conv = pd.concat(data_conv, axis=1)
del convertion

display(data_conv.head(15))


11676
X_test_intern post::  (11676, 1)
X_test_intern post::  (11676, 1)
X_test_intern post::  (11676, 1)
X_test_intern post::  (11676, 1)


,TARGET_1,TARGET_2,TARGET_3,TARGET_4
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [15]:

current_col = 'current_convertion'
previous_col = 'previous_convertion'
before_previous_col = 'before_previous_convertion'

X_train[current_col] = data_conv['TARGET_3']
X_train[previous_col] = data_conv['TARGET_2']
X_train[before_previous_col] = data_conv['TARGET_1']

X_test[current_col] = data_conv['TARGET_4']
X_test[previous_col] = data_conv['TARGET_3']
X_test[before_previous_col] = data_conv['TARGET_2']

del data_conv


In [16]:
X_train.shape, X_test.shape, y_train.shape

((11387, 1948), (11676, 1948), (11387,))

In [17]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [500],
    'feature_fraction': [0.4],
    'bagging_fraction': [0.8],
    'max_depth':[55],
    'learning_rate': [0.05]
}
    
kfold_off = StratifiedKFold(
    n_splits=10, 
    shuffle=False, 
    random_state=SEED
)
model_lgbm = LGBMClassifier(seed=SEED)

grid = GridSearchCV(
    model_lgbm, param_grid, cv=kfold_off, 
    n_jobs=-1, scoring='roc_auc', verbose=1
)
grid.fit(X_train, y_train)

best_params_ = grid.best_params_
model_ = grid.best_estimator_
score_ = grid.best_score_
print(score_, best_params_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.6min finished


0.8001376861053056 {'bagging_fraction': 0.8, 'feature_fraction': 0.4, 'learning_rate': 0.05, 'max_depth': 55, 'n_estimators': 500}


In [18]:
predict_test = grid.best_estimator_.predict_proba(X_test)[:,1]
print(predict_test, predict_test.shape)
predict_test = pd.DataFrame(
    {'USER_ID':X_test.index, 'SCORE':predict_test} 
)
predict_test.head()

[2.06738960e-04 1.12786108e-04 1.91262875e-05 ... 5.39646384e-05
 4.37300033e-05 4.60241062e-05] (11676,)


,USER_ID,SCORE
0,0,0.000207
1,1,0.000113
2,2,0.000019
3,3,0.000043
4,4,0.001177


In [19]:
predict_test.shape

(11676, 2)

In [20]:
"""
idx_faltantes = list(set(X_train.index) - set(predict_test['USER_ID'].unique()))
len(idx_faltantes)
"""

"\nidx_faltantes = list(set(X_train.index) - set(predict_test['USER_ID'].unique()))\nlen(idx_faltantes)\n"

In [21]:
"""
faltantes = pd.DataFrame({
    'USER_ID':idx_faltantes, 'SCORE': [y_train_2[_] for _ in idx_faltantes]
})
faltantes.head()
"""

"\nfaltantes = pd.DataFrame({\n    'USER_ID':idx_faltantes, 'SCORE': [y_train_2[_] for _ in idx_faltantes]\n})\nfaltantes.head()\n"

In [22]:
"""
predict_test = pd.concat([predict_test, faltantes], axis=0).sort_values(['USER_ID'])
predict_test.shape
"""

"\npredict_test = pd.concat([predict_test, faltantes], axis=0).sort_values(['USER_ID'])\npredict_test.shape\n"

In [24]:
predict_test.to_csv('fix_brenchmark_again.csv_test_3trimestres_beforev.csv', index=False)